# Business Market Analysis
___

## Introduction / Business Problem

___

This analysis is meant to help a business venture choose between locations. Specifically we will look at Mexican retaraunts in Los Angeles. Finding a neighborhood that is not saturated with other Mexican restaunts would be a great place to start!

## Data

___

### Imports and authentication

___

In [11]:
import numpy as np
import pandas as pd
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import requests
import urllib.request
from urllib.request import urlopen
import time
from bs4 import BeautifulSoup
!pip install pgeocode
import pgeocode
from pandas_profiling import ProfileReport
!pip install sweetviz
import sweetviz as sv
!pip install autoviz
import plotly.express as px

<div class="alert alert-block alert-danger">
<b>Delete this cell before publishing
</div>



In [12]:
# Foursquare API authentication
CLIENT_ID = '' 
CLIENT_SECRET = '' 
ACCESS_TOKEN = '' 
VERSION = '20180604'

___

### scrape for list of neighborhoods in LA


___

First we need to set some variables, including defining the source of our information. We will be scraping LAalminac.com to get out communities and zip codes.

In [13]:
url = "http://www.laalmanac.com/communications/cm02_communities.php" 
data = requests.get(url).text 
soup = BeautifulSoup(data,"html.parser")

Here we are creating a list, finding our cells, cleaning some unnececary characters, and appending our list.

In [14]:
import re

table=soup.find('table')
list_rows = []
for row in table:
    cells = soup.find_all('td')
    str_cells = str(cells)
    clean = re.compile('<.*?>')
    clean2 = (re.sub(clean, '',str_cells))
    list_rows.append(clean2)
print(clean2)
type(clean2)

[Acton, 93510, Agoura Hills, 91301, Agoura Hills (PO Boxes), 91376, Agua Dulce, 91390, Alhambra, 91801, 91803, Alhambra (Non-Geographic Zip Code Within 91801), 91804, Alhambra (PO Boxes), 91802, 91896, 91899, Altadena, 91001, Altadena (PO Boxes), 91003, Americana at Brand &amp; Glendale Galleria (Glendale), 91210, Arcadia, 91006, 91007, Arcadia (PO Boxes), 91066, 91077, Arleta (Los Angeles), 91331, Arlington Heights (Los Angeles), 90019, Artesia, 90701, Artesia (PO Boxes), 90702, Athens, 90044, Atwater Village (Los Angeles), 90039, Avalon (PO Boxes), 90704, Avocado Heights, 91746, Azusa, 91010, 91702, Baldwin Hills (Los Angeles), 90008, Baldwin Park, 91706, Bassett, 91746, Bel Air Estates (Los Angeles), 90049, Bel Air Estates, Beverly Glen (Los Angeles), 90077, Bell, 90201, 90270, Bell (PO Boxes), 90202, Bell (Shared Firms), 90096, Bell Canyon, 91307, Bell Gardens, 90201, Bell Gardens (PO Boxes), 90202, Bellflower, 90706, Bellflower (PO Boxes), 90707, Belmont Shore (Long Beach), 90803,

str

Next we must split the data at the commas.

In [15]:
s = clean2.split(',')
# Uncomment for error catching
# s

Here we have the bulk of our data cleaning! See the comments for more information on what is happening but the basic idea is to take a list of strings and turn some of the items into int, while also getting rid of some extra zip codes we wont be using and fix a couple formatting problems. We will also be grabbing some latitude and longitude data while we are at it

In [16]:
#remove spaces
s1 = [s.strip(' ') for s in s]
#create empty list
a = []
nomi = pgeocode.Nominatim('US')
#create variable to use for itteration
cityNumber = 0
#create boolean variables for determining if last itterated was a string or int
lastWasZIP = False
lastwasletter = False
#for loop to convert zip codes to int and group city with zip code
for x in s1:
  if not x[1].isdigit() and lastwasletter == False: #if current item is NOT a number AND last item WAS
    if cityNumber == 0:                                 #if city number is 0->
      a.append([x[1:],10000,0,0])                           #append with 10000 for easy to spot error catching and remove 1st "["
      cityNumber = cityNumber + 1                       #move to next variable
      lastwasletter = True                              #last item WAS a letter
    else:
      if not str(a[cityNumber-1][0][0:5]) in x:         #if not a str that matches previous cities 1st 6 characters
        a.append([x,10000,0,0])                             #append with 10000 for easy to spot error catching
        cityNumber = cityNumber + 1                     #move to next variable
        lastWasZIP = False                              #last item was NOT a number
        lastwasletter = True                            #last item WAS a letter
  elif lastWasZIP == False and x[1].isdigit():      #if last item was NOT a number AND current IS
      if x[len(x)-1] != "]":                            
        a[cityNumber-1][1] = int(x)                     #replace 10000 with int version or this line of x     
        a[cityNumber-1][2] = nomi.query_postal_code(int(x)).latitude
        a[cityNumber-1][3] = nomi.query_postal_code(int(x)).longitude
        lastWasZIP = True
        lastwasletter = False
      else:
        a[cityNumber-1][1] = int(x[:-1])                #replace 10000 with int version without last character of x
        a[cityNumber-1][2] = nomi.query_postal_code(int(x[:-1])).latitude
        a[cityNumber-1][3] = nomi.query_postal_code(int(x[:-1])).longitude
        lastWasZIP = True
        lastwasletter = False

In [17]:
# Uncomment for error catching
#a

Lets make it a data frame and have a look!

In [18]:
df = pd.DataFrame(a, columns =['Neighborhood', 'PostalCode', 'Latitude', 'Longitude'])
df

,Neighborhood,PostalCode,Latitude,Longitude
0,Acton,93510,34.4835,-118.1959
1,Agoura Hills,91301,34.1227,-118.7573
2,Agua Dulce,91390,34.4684,-118.5261
3,Alhambra,91801,34.0914,-118.1293
4,Altadena,91001,34.1912,-118.1392
...,...,...,...,...
256,Wilshire Center (Los Angeles),90004,34.0762,-118.3029
257,Wilsona Gardens,93535,34.7131,-117.8783
258,Windsor Hills (Los Angeles),90043,33.9871,-118.3321
259,Winnetka (Los Angeles),91306,34.2092,-118.5749


___

### define a function to locate nearby venues
___


This will grab all the restaurants labeled "Mexican"

In [19]:
LIMIT = 50
openNow = 0
sortByDistance = 0

def getNearbyVenues(Neighborhood, PostalCode, Latitude, Longitude, radius=500):
    
    venues_list=[]
    for name, postalcode, lat, lng in zip(Neighborhood, PostalCode, Latitude, Longitude):
        if postalcode != 91797:
          url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
              CLIENT_ID, 
              CLIENT_SECRET, 
              VERSION, 
              lat, 
              lng, 
              radius, 
              LIMIT,)
            
          results = requests.get(url).json()["response"]['groups'][0]['items']
          for v in results:
            if v['venue']['categories'][0]['shortName'] == 'Mexican':
              venues_list.append([(
               name, 
               lat, 
               lng, 
               v['venue']['id'],
               v['venue']['name'], 
               v['venue']['location']['lat'], 
               v['venue']['location']['lng'],  
               v['venue']['categories'][0]['name'])])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Lat',
                  'Neighborhood Long',
                  'ID',
                  'Venue', 
                  'Venue Lat',
                  'Venue Long',
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
LA_venues = getNearbyVenues(Neighborhood=df['Neighborhood'],PostalCode=df['PostalCode'], Latitude=df['Latitude'],Longitude=df['Longitude'])

In [21]:
LA_venues

,Neighborhood,Neighborhood Lat,Neighborhood Long,ID,Venue,Venue Lat,Venue Long,Venue Category
0,Arcadia,34.1324,-118.0264,4b42664ef964a52023d325e3,Taco Lita,34.130003,-118.027064,Mexican Restaurant
1,Arlington Heights (Los Angeles),34.0482,-118.3343,524b20ba11d298598ce1c917,Chipotle Mexican Grill,34.048183,-118.336012,Mexican Restaurant
2,Arlington Heights (Los Angeles),34.0482,-118.3343,4aeb7b84f964a52088c221e3,El Compita,34.048592,-118.332846,Mexican Restaurant
3,Azusa,34.1407,-117.9567,4bca374f511f95219e57afc7,Tacos Ensenada,34.140143,-117.957711,Mexican Restaurant
4,Baldwin Hills (Los Angeles),34.0116,-118.3411,50f6caa4b0b9aa3a48bead1d,Chipotle Mexican Grill,34.013284,-118.336625,Mexican Restaurant
...,...,...,...,...,...,...,...,...
192,Willowbrook,33.9293,-118.2463,4d61d133e4fe54815b25909e,Tacos El Bronco,33.929620,-118.246058,Mexican Restaurant
193,Wilmington (Los Angeles),33.7855,-118.2645,4bb64c126edc76b04cb6301c,Mariscos La Paz,33.787514,-118.263280,Mexican Restaurant
194,Wilshire Center (Los Angeles),34.0762,-118.3029,49e8c872f964a5207c651fe3,Cactus Mexican Food,34.076194,-118.304147,Mexican Restaurant
195,Wilshire Center (Los Angeles),34.0762,-118.3029,4c2932e49eb1952105ad2959,Casa Carnitas,34.076540,-118.297723,Mexican Restaurant


___

### map venues in relation to neighborhoods

___

Here is an interactive map that shows all the venues in blue and all the neighborhoods in red.

In [22]:
map_LA = folium.Map(location=[34.1324,-118.0264],zoom_start=10)

for ven_lat,ven_lng,neighborhood, venue in zip(LA_venues['Venue Lat'],LA_venues['Venue Long'],LA_venues['Neighborhood'],LA_venues['Venue']):
    label = '{}, {}'.format(venue, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [ven_lat,ven_lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_LA)

for n_lat,n_lng,neighborhood in zip(LA_venues['Neighborhood Lat'],LA_venues['Neighborhood Long'],LA_venues['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [n_lat,n_lng],
    radius=3,
    popup=label,
    color='red',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.4,
    parse_html=False).add_to(map_LA)

map_LA

In [23]:
count = (LA_venues.groupby(['Neighborhood']).size() 
   .sort_values(ascending=False) 
   .reset_index(name='count')) 
count

,Neighborhood,count
0,Downtown Arts District (Los Angeles),6
1,Chinatown (Los Angeles),6
2,Mission Hills (Los Angeles),5
3,San Fernando,5
4,Sylmar (Los Angeles),5
...,...,...
104,La Tuna Canyon (Los Angeles),1
105,La Habra Heights,1
106,Koreatown (Los Angeles),1
107,Irwindale,1


___

### Pick a nieghborhood

___

Wile I dont want to pick an area that is saturated I also don't want to pick an area with no demand for a Mexican Restaruant. Since I like the neighborhood University Park and it has enough venues to show demand lets look a little deeper!

In [24]:
count.loc[count['Neighborhood'] == 'University Park (Los Angeles)']

,Neighborhood,count
39,University Park (Los Angeles),2


In [25]:
UP_venues = LA_venues.loc[LA_venues['Neighborhood'] == 'University Park (Los Angeles)']
UP_venues.reset_index
UP_venues

,Neighborhood,Neighborhood Lat,Neighborhood Long,ID,Venue,Venue Lat,Venue Long,Venue Category
172,University Park (Los Angeles),34.0294,-118.2871,4fac7382e4b021d7498d6385,La Valentina,34.029765,-118.291645,Mexican Restaurant
173,University Park (Los Angeles),34.0294,-118.2871,5179acca498e4dd185fd3b87,Monarca,34.030336,-118.291802,Mexican Restaurant


___

### La Valentina

___

![alt text](https://fastly.4sqi.net/img/general/717x959/58142653_JJ4pV1PYqgFfdh0JzpQhKRMUq63Tv2zqCNrOdmgVHao.jpg "Title")

In [26]:
venue_id = UP_venues.iloc[0][3]
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(
    venue_id, 
    CLIENT_ID, 
    CLIENT_SECRET,
    ACCESS_TOKEN, 
    VERSION
    )
result = requests.get(url).json()

In [27]:
# Uncomment for error testing
# result

In [28]:
# See what responses I can pull
result['response']['venue'].keys()

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'likes', 'like', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'seasonalHours', 'defaultHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])

In [29]:
# Uncomment for error testing
# result['response']['venue']

In [30]:
result['response']['venue']['rating']

7.0

In [31]:
result['response']['venue']['likes']['count']

14

In [32]:
result['response']['venue']['hours']

{'dayData': [],
 'isLocalHoliday': False,
 'isOpen': False,
 'richStatus': {'entities': [], 'text': 'Closed until 8:00 AM'},
 'status': 'Closed until 8:00 AM',
 'timeframes': [{'days': 'Mon–Sun',
   'includesToday': True,
   'open': [{'renderedTime': '8:00 AM–9:00 PM'}],
   'segments': []}]}

In [33]:
result['response']['venue']['popular']

{'isLocalHoliday': False,
 'isOpen': False,
 'timeframes': [{'days': 'Today',
   'includesToday': True,
   'open': [{'renderedTime': 'Noon–7:00 PM'}],
   'segments': []},
  {'days': 'Tue',
   'open': [{'renderedTime': '10:00 AM–9:00 PM'}],
   'segments': []},
  {'days': 'Wed', 'open': [{'renderedTime': 'Noon–5:00 PM'}], 'segments': []},
  {'days': 'Thu',
   'open': [{'renderedTime': 'Noon–5:00 PM'},
    {'renderedTime': '8:00 PM–9:00 PM'}],
   'segments': []},
  {'days': 'Fri',
   'open': [{'renderedTime': '10:00 AM–3:00 PM'},
    {'renderedTime': '5:00 PM–10:00 PM'}],
   'segments': []},
  {'days': 'Sat',
   'open': [{'renderedTime': '9:00 AM–9:00 PM'}],
   'segments': []},
  {'days': 'Sun',
   'open': [{'renderedTime': '9:00 AM–10:00 PM'}],
   'segments': []}]}

In [34]:
result['response']['venue']['stats']
# This does not contain usersCount or checkinsCount

{'tipCount': 6}

In [35]:
result['response']['venue']['url']

'http://lavalentinamexicangrill.com'

In [36]:
result['response']['venue']['price']

{'currency': '$', 'message': 'Moderate', 'tier': 2}

In [37]:
result['response']['venue']['createdAt']

1336701826

In [38]:
result['response']['venue']['likes']


{'count': 14,
 'groups': [{'count': 14, 'items': [], 'type': 'others'}],
 'summary': '14 Likes'}

Lets pull some similar venues to get a feel for the place

In [39]:
urlx = 'https://api.foursquare.com/v2/venues/{}/similar?client_id={}&client_secret={}&oauth_token={}&v={}'.format(
    venue_id, 
    CLIENT_ID, 
    CLIENT_SECRET,
    ACCESS_TOKEN, 
    VERSION
    )
resultx = requests.get(urlx).json()

In [40]:
# resultx['response']['similarVenues']['items']
similar = pd.json_normalize(resultx['response']['similarVenues']['items'])
similar 

,id,name,categories,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,venuePage.id
0,4a872cecf964a520090320e3,Chipotle Mexican Grill,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",3748 S Figueroa St,at State Dr,34.016956,-118.282584,"[{'label': 'display', 'lat': 34.01695555547079...",90007,US,Los Angeles,CA,United States,"[3748 S Figueroa St (at State Dr), Los Angeles...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,49f414b9f964a520fa6a1fe3,El Taurino,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",1104 S Hoover St,W 11th St,34.051589,-118.283891,"[{'label': 'display', 'lat': 34.05158867766603...",90006,US,Los Angeles,CA,United States,"[1104 S Hoover St (W 11th St), Los Angeles, CA...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,450e92b1f964a52063391fe3,La Taquiza USC,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",3009 S Figueroa St,at W 30th St.,34.025061,-118.278685,"[{'label': 'display', 'lat': 34.02506138329362...",90007,US,Los Angeles,CA,United States,"[3009 S Figueroa St (at W 30th St.), Los Angel...",1393373,https://www.grubhub.com/restaurant/la-taquiza-...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,50028218
3,4baebd61f964a520c6d33be3,Chichen Itza Restaurant,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",3655 S Grand Ave #C6,inside Mercado La Paloma,34.017489,-118.278484,"[{'label': 'display', 'lat': 34.01748875363889...",90007,US,Los Angeles,CA,United States,[3655 S Grand Ave #C6 (inside Mercado La Palom...,2485925,https://www.grubhub.com/restaurant/chichen-itz...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,43631373
4,4c268810136d20a1d78ce561,Oaxacalifornia,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",3655 S Grand Ave,NaN,34.017444,-118.278372,"[{'label': 'display', 'lat': 34.01744369853196...",90007,US,Los Angeles,CA,United States,"[3655 S Grand Ave, Los Angeles, CA 90007]",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
#index through id column to create variables for a call

In [42]:
result['response']['venue']['bestPhoto']

{'createdAt': 1379443191,
 'height': 717,
 'id': '5238a1f7498e93328a0d057a',
 'prefix': 'https://fastly.4sqi.net/img/general/',
 'source': {'name': 'Foursquare for iOS',
  'url': 'https://foursquare.com/download/#/iphone'},
 'suffix': '/58142653_JJ4pV1PYqgFfdh0JzpQhKRMUq63Tv2zqCNrOdmgVHao.jpg',
 'visibility': 'public',
 'width': 959}

In [43]:
a = result['response']['venue']['bestPhoto']['prefix']
b = str(result['response']['venue']['bestPhoto']['height'])
c = 'x'
d = str(result['response']['venue']['bestPhoto']['width'])
e = result['response']['venue']['bestPhoto']['suffix']

In [44]:
a + b + c + d + e

'https://fastly.4sqi.net/img/general/717x959/58142653_JJ4pV1PYqgFfdh0JzpQhKRMUq63Tv2zqCNrOdmgVHao.jpg'

___

### Monarca

___

![alt text](https://fastly.4sqi.net/img/general/1440x1440/2705098_J9kyd0opR05-Nt2wySC2cukfZNocqosrCzXnLp9suZE.jpg "Title")

In [45]:
venue_id1 = UP_venues.iloc[1][3]
url1 = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(
    venue_id1, 
    CLIENT_ID, 
    CLIENT_SECRET,
    ACCESS_TOKEN, 
    VERSION
    )
result1 = requests.get(url1).json()

In [46]:
# Verify I have the correct venue
venue_id1

'5179acca498e4dd185fd3b87'

In [47]:
# Uncomment for error testing
# result1['response']['venue']

In [48]:
result1['response']['venue'].keys()

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'price', 'likes', 'like', 'dislike', 'ok', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'seasonalHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])

In [49]:
try:
    print(result1['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [50]:
try:
    print(result1['response']['venue']['hours'])
except:
    print('This venue has no hours information.')

This venue has no hours information.


In [51]:
try:
    print(result1['response']['venue']['popular'])
except:
    print('This venue has no popularity information.')

This venue has no popularity information.


In [52]:
result1['response']['venue']['likes']['count']

1

In [53]:
result1['response']['venue']['stats']

{'tipCount': 0}

In [54]:
try:
  result1['response']['venue']['url']
except:
  print("NaN")

NaN


In [55]:
result1['response']['venue']['price']

{'currency': '$', 'message': 'Cheap', 'tier': 1}

In [56]:
result1['response']['venue']['createdAt']

1366928586

In [57]:
result1['response']['venue']['likes']


{'count': 1,
 'groups': [{'count': 1,
   'items': [{'countryCode': 'US',
     'firstName': 'Reynaldo',
     'gender': 'male',
     'id': '391656528',
     'lastName': 'Sosa',
     'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
      'suffix': '/391656528_1A5vi2C1_MB48N510CnIhJRgBImlN6ci5qQto6pFO6ParF1rdNjzO9EI8ztW1jw6XGIiOn4cY.jpg'}}],
   'type': 'others'}],
 'summary': 'Reynaldo Sosa'}

Lets pull some similar venues to get a feel for the place

In [58]:
urlx1 = 'https://api.foursquare.com/v2/venues/{}/similar?client_id={}&client_secret={}&oauth_token={}&v={}'.format(
    venue_id, 
    CLIENT_ID, 
    CLIENT_SECRET,
    ACCESS_TOKEN, 
    VERSION
    )
resultx1 = requests.get(urlx1).json()

In [59]:
# resultx['response']['similarVenues']['items']
similar1 = pd.json_normalize(resultx1['response']['similarVenues']['items'])
similar1 

,id,name,categories,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,venuePage.id
0,4a872cecf964a520090320e3,Chipotle Mexican Grill,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",3748 S Figueroa St,at State Dr,34.016956,-118.282584,"[{'label': 'display', 'lat': 34.01695555547079...",90007,US,Los Angeles,CA,United States,"[3748 S Figueroa St (at State Dr), Los Angeles...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,49f414b9f964a520fa6a1fe3,El Taurino,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",1104 S Hoover St,W 11th St,34.051589,-118.283891,"[{'label': 'display', 'lat': 34.05158867766603...",90006,US,Los Angeles,CA,United States,"[1104 S Hoover St (W 11th St), Los Angeles, CA...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,450e92b1f964a52063391fe3,La Taquiza USC,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",3009 S Figueroa St,at W 30th St.,34.025061,-118.278685,"[{'label': 'display', 'lat': 34.02506138329362...",90007,US,Los Angeles,CA,United States,"[3009 S Figueroa St (at W 30th St.), Los Angel...",1393373,https://www.grubhub.com/restaurant/la-taquiza-...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,50028218
3,4baebd61f964a520c6d33be3,Chichen Itza Restaurant,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",3655 S Grand Ave #C6,inside Mercado La Paloma,34.017489,-118.278484,"[{'label': 'display', 'lat': 34.01748875363889...",90007,US,Los Angeles,CA,United States,[3655 S Grand Ave #C6 (inside Mercado La Palom...,2485925,https://www.grubhub.com/restaurant/chichen-itz...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,43631373
4,4c268810136d20a1d78ce561,Oaxacalifornia,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",3655 S Grand Ave,NaN,34.017444,-118.278372,"[{'label': 'display', 'lat': 34.01744369853196...",90007,US,Los Angeles,CA,United States,"[3655 S Grand Ave, Los Angeles, CA 90007]",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
result1['response']['venue']['bestPhoto']

{'createdAt': 1484376519,
 'height': 1440,
 'id': '5879c9c77b88a705eb7a97bf',
 'prefix': 'https://fastly.4sqi.net/img/general/',
 'source': {'name': 'Swarm for iOS', 'url': 'https://www.swarmapp.com'},
 'suffix': '/2705098_J9kyd0opR05-Nt2wySC2cukfZNocqosrCzXnLp9suZE.jpg',
 'visibility': 'public',
 'width': 1440}

In [61]:
a1 = result1['response']['venue']['bestPhoto']['prefix']
b1 = str(result1['response']['venue']['bestPhoto']['height'])
c1 = 'x'
d1 = str(result1['response']['venue']['bestPhoto']['width'])
e1 = result1['response']['venue']['bestPhoto']['suffix']

In [62]:
a1 + b1 + c1 + d1 + e1

'https://fastly.4sqi.net/img/general/1440x1440/2705098_J9kyd0opR05-Nt2wySC2cukfZNocqosrCzXnLp9suZE.jpg'

Here is the "Best Photo" from the venue:

___

### Cilantro Mexican Grill

___

This restaurant did not populate this time so we will comment this out for now.

In [63]:
# venue_id2 = UP_venues.iloc[2][3]
# url2 = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)

In [64]:
# venue_id2

In [65]:
# result2 = requests.get(url2).json()
# print(result2['response']['venue'].keys())
# result2['response']['venue']

___

### Dataframe to compare restaraunts

___

In [66]:
dfx = pd.json_normalize(result['response']['venue'])

In [67]:
dfx

,id,name,canonicalUrl,categories,verified,url,like,dislike,ok,rating,ratingColor,ratingSignals,allowMenuUrlEdit,createdAt,shortUrl,timeZone,seasonalHours,contact.phone,contact.formattedPhone,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,stats.tipCount,price.tier,price.message,price.currency,likes.count,likes.groups,likes.summary,beenHere.count,beenHere.unconfirmedCount,beenHere.marked,...,tips.count,tips.groups,listed.count,listed.groups,hours.status,hours.richStatus.entities,hours.richStatus.text,hours.isOpen,hours.isLocalHoliday,hours.dayData,hours.timeframes,popular.isOpen,popular.isLocalHoliday,popular.timeframes,defaultHours.status,defaultHours.richStatus.entities,defaultHours.richStatus.text,defaultHours.isOpen,defaultHours.isLocalHoliday,defaultHours.dayData,defaultHours.timeframes,pageUpdates.count,pageUpdates.items,inbox.count,inbox.items,attributes.groups,bestPhoto.id,bestPhoto.createdAt,bestPhoto.source.name,bestPhoto.source.url,bestPhoto.prefix,bestPhoto.suffix,bestPhoto.width,bestPhoto.height,bestPhoto.visibility,colors.highlightColor.photoId,colors.highlightColor.value,colors.highlightTextColor.photoId,colors.highlightTextColor.value,colors.algoVersion
0,4fac7382e4b021d7498d6385,La Valentina,https://foursquare.com/v/la-valentina/4fac7382...,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",False,http://lavalentinamexicangrill.com,False,False,False,7.0,C5DE35,22,True,1336701826,http://4sq.com/JE0N2l,America/Los_Angeles,[],3237344071,(323) 734-4071,2719 S Vermont Ave,S Vermont And W 28 St,34.029765,-118.291645,"[{'label': 'display', 'lat': 34.02976458170659...",90007,US,Los Angeles,CA,United States,"[2719 S Vermont Ave (S Vermont And W 28 St), L...",6,2,Moderate,$,14,"[{'type': 'others', 'count': 14, 'items': []}]",14 Likes,0,0,False,...,6,"[{'type': 'following', 'name': 'Tips from peop...",4,"[{'type': 'others', 'name': 'Lists from other ...",Closed until 8:00 AM,[],Closed until 8:00 AM,False,False,[],"[{'days': 'Mon–Sun', 'includesToday': True, 'o...",False,False,"[{'days': 'Today', 'includesToday': True, 'ope...",Closed until 8:00 AM,[],Closed until 8:00 AM,False,False,[],"[{'days': 'Mon–Sun', 'includesToday': True, 'o...",0,[],0,[],"[{'type': 'price', 'name': 'Price', 'summary':...",5238a1f7498e93328a0d057a,1379443191,Foursquare for iOS,https://foursquare.com/download/#/iphone,https://fastly.4sqi.net/img/general/,/58142653_JJ4pV1PYqgFfdh0JzpQhKRMUq63Tv2zqCNrO...,959,717,public,5238a1f7498e93328a0d057a,-3092296,5238a1f7498e93328a0d057a,-16777216,3


In [68]:
dfxx = dfx[[
            'id',
            'name',            
            'verified',
            'likes.count',            
            'rating',
            'ratingSignals',
            'stats.tipCount',
            'price.tier',
            'price.message',
            'createdAt',
            'contact.formattedPhone',
            'location.address',
            'url',            
            ]]

In [69]:
dfxx

,id,name,verified,likes.count,rating,ratingSignals,stats.tipCount,price.tier,price.message,createdAt,contact.formattedPhone,location.address,url
0,4fac7382e4b021d7498d6385,La Valentina,False,14,7.0,22,6,2,Moderate,1336701826,(323) 734-4071,2719 S Vermont Ave,http://lavalentinamexicangrill.com



<div class="alert alert-block alert-info">
<b>Note:</b> This brings back slightly different information from the section above where we call the json file directly. Example: best photo is different
</div>

### Lets pull some similar venues to get a feel for the place

In [70]:
urlx = 'https://api.foursquare.com/v2/venues/{}/similar?client_id={}&client_secret={}&oauth_token={}&v={}'.format(
    venue_id, 
    CLIENT_ID, 
    CLIENT_SECRET,
    ACCESS_TOKEN, 
    VERSION
    )
resultx = requests.get(urlx).json()

In [71]:
# resultx['response']['similarVenues']['items']
similar = pd.json_normalize(resultx['response']['similarVenues']['items'])
similar

,id,name,categories,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,venuePage.id
0,4a872cecf964a520090320e3,Chipotle Mexican Grill,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",3748 S Figueroa St,at State Dr,34.016956,-118.282584,"[{'label': 'display', 'lat': 34.01695555547079...",90007,US,Los Angeles,CA,United States,"[3748 S Figueroa St (at State Dr), Los Angeles...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,49f414b9f964a520fa6a1fe3,El Taurino,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",1104 S Hoover St,W 11th St,34.051589,-118.283891,"[{'label': 'display', 'lat': 34.05158867766603...",90006,US,Los Angeles,CA,United States,"[1104 S Hoover St (W 11th St), Los Angeles, CA...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,450e92b1f964a52063391fe3,La Taquiza USC,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",3009 S Figueroa St,at W 30th St.,34.025061,-118.278685,"[{'label': 'display', 'lat': 34.02506138329362...",90007,US,Los Angeles,CA,United States,"[3009 S Figueroa St (at W 30th St.), Los Angel...",1393373,https://www.grubhub.com/restaurant/la-taquiza-...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,50028218
3,4baebd61f964a520c6d33be3,Chichen Itza Restaurant,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",3655 S Grand Ave #C6,inside Mercado La Paloma,34.017489,-118.278484,"[{'label': 'display', 'lat': 34.01748875363889...",90007,US,Los Angeles,CA,United States,[3655 S Grand Ave #C6 (inside Mercado La Palom...,2485925,https://www.grubhub.com/restaurant/chichen-itz...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,43631373
4,4c268810136d20a1d78ce561,Oaxacalifornia,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",3655 S Grand Ave,NaN,34.017444,-118.278372,"[{'label': 'display', 'lat': 34.01744369853196...",90007,US,Los Angeles,CA,United States,"[3655 S Grand Ave, Los Angeles, CA 90007]",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
similar['id']

0    4a872cecf964a520090320e3
1    49f414b9f964a520fa6a1fe3
2    450e92b1f964a52063391fe3
3    4baebd61f964a520c6d33be3
4    4c268810136d20a1d78ce561
Name: id, dtype: object

In [73]:
similar['id'].iteritems

<bound method Series.iteritems of 0    4a872cecf964a520090320e3
1    49f414b9f964a520fa6a1fe3
2    450e92b1f964a52063391fe3
3    4baebd61f964a520c6d33be3
4    4c268810136d20a1d78ce561
Name: id, dtype: object>

In [74]:
similar['id'][0]

'4a872cecf964a520090320e3'

In [75]:
venue_idy = similar['id'].iteritems()
venue_idy

In [76]:
similar_venues = []

for venue_idz in similar['id']:

  urly = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(
    venue_idz, 
    CLIENT_ID, 
    CLIENT_SECRET,
    ACCESS_TOKEN, 
    VERSION
    )

  listy = []
  x = requests.get(urly).json()['response']['venue']

  try:
    listy.append(str(x['id']))
  except: 
    listy.append(None)
  try:   
    listy.append(str(x['name']))
  except: 
    listy.append(None)
  try:
    listy.append(str(x['verified']))
  except: 
    listy.append(None)
  try:
    listy.append(str(x['likes']['count']))
  except: 
    listy.append(None)
  try:
    listy.append(str(x['rating']))
  except: 
    listy.append(None)
  try:
    listy.append(str(x['ratingSignals']))
  except: 
    listy.append(None)
  try:
    listy.append(str(x['stats']['tipCount']))
  except: 
    listy.append(None)
  try:
    listy.append(str(x['price']['tier']))
  except: 
    listy.append(None)
  try:
    listy.append(str(x['price']['message']))
  except: 
    listy.append(None)
  try:
    listy.append(str(x['createdAt']))
  except: 
    listy.append(None)
  try:
    listy.append(str(x['contact']['formattedPhone']))
  except: 
    listy.append(None)
  try:
    listy.append(str(x['location']['address']))
  except: 
    listy.append(None)
  try:
    listy.append(str(x['url']))
  except: 
    listy.append(None)
  
  similar_venues.append(listy)
  


In [77]:
listy

['4c268810136d20a1d78ce561',
 'Oaxacalifornia',
 'False',
 '1',
 None,
 None,
 '0',
 '1',
 'Cheap',
 '1277593616',
 None,
 '3655 S Grand Ave',
 None]

In [78]:
sim_ven = pd.DataFrame(similar_venues)
sim_ven.columns = ['id','name','verified','likes.count','rating','ratingSignals','stats.tipCount','price.tier','price.message','createdAt','contact.formattedPhone','location.address','url']

In [79]:
dfxx


,id,name,verified,likes.count,rating,ratingSignals,stats.tipCount,price.tier,price.message,createdAt,contact.formattedPhone,location.address,url
0,4fac7382e4b021d7498d6385,La Valentina,False,14,7.0,22,6,2,Moderate,1336701826,(323) 734-4071,2719 S Vermont Ave,http://lavalentinamexicangrill.com


In [80]:
dfxxx = dfxx.append(sim_ven)
dfxxx

,id,name,verified,likes.count,rating,ratingSignals,stats.tipCount,price.tier,price.message,createdAt,contact.formattedPhone,location.address,url
0,4fac7382e4b021d7498d6385,La Valentina,False,14,7,22,6,2,Moderate,1336701826,(323) 734-4071,2719 S Vermont Ave,http://lavalentinamexicangrill.com
0,4a872cecf964a520090320e3,Chipotle Mexican Grill,True,92,7.9,127,26,2,Moderate,1250372844,(213) 765-9068,3748 S Figueroa St,https://locations.chipotle.com/ca/los-angeles/...
1,49f414b9f964a520fa6a1fe3,El Taurino,False,161,8.0,275,116,1,Cheap,1240732857,(213) 738-9197,1104 S Hoover St,http://www.kingtaco.com/locations/store_0105.html
2,450e92b1f964a52063391fe3,La Taquiza USC,True,39,7.7,73,32,2,Moderate,1158582961,(213) 741-9795,3009 S Figueroa St,http://lataquizaktown.com
3,4baebd61f964a520c6d33be3,Chichen Itza Restaurant,True,176,8.9,233,63,1,Cheap,1269742945,(213) 741-1075,3655 S Grand Ave #C6,http://www.chichenitzarestaurant.com
4,4c268810136d20a1d78ce561,Oaxacalifornia,False,1,None,None,0,1,Cheap,1277593616,None,3655 S Grand Ave,None


In [81]:
dfxxx.reset_index(inplace=True, drop=True)

dfxxx

,id,name,verified,likes.count,rating,ratingSignals,stats.tipCount,price.tier,price.message,createdAt,contact.formattedPhone,location.address,url
0,4fac7382e4b021d7498d6385,La Valentina,False,14,7,22,6,2,Moderate,1336701826,(323) 734-4071,2719 S Vermont Ave,http://lavalentinamexicangrill.com
1,4a872cecf964a520090320e3,Chipotle Mexican Grill,True,92,7.9,127,26,2,Moderate,1250372844,(213) 765-9068,3748 S Figueroa St,https://locations.chipotle.com/ca/los-angeles/...
2,49f414b9f964a520fa6a1fe3,El Taurino,False,161,8.0,275,116,1,Cheap,1240732857,(213) 738-9197,1104 S Hoover St,http://www.kingtaco.com/locations/store_0105.html
3,450e92b1f964a52063391fe3,La Taquiza USC,True,39,7.7,73,32,2,Moderate,1158582961,(213) 741-9795,3009 S Figueroa St,http://lataquizaktown.com
4,4baebd61f964a520c6d33be3,Chichen Itza Restaurant,True,176,8.9,233,63,1,Cheap,1269742945,(213) 741-1075,3655 S Grand Ave #C6,http://www.chichenitzarestaurant.com
5,4c268810136d20a1d78ce561,Oaxacalifornia,False,1,None,None,0,1,Cheap,1277593616,None,3655 S Grand Ave,None


In [82]:

dfxxx['verified'] = dfxxx['verified'].astype(str) 
# for verified in dfxxx['verified']:
#   if dfxxx['verified'].str.contains("True"):
#     dfxxx['verified'] = 1
#   else:
#     dfxxx['verified'] = 0   
dfxxx['likes.count'] = pd.to_numeric(dfxxx['likes.count'], errors='coerce')
dfxxx['createdAt'] = dfxxx['createdAt'].astype(int)
dfxxx['rating'] = pd.to_numeric(dfxxx['rating'], errors='coerce')
dfxxx['ratingSignals'] = pd.to_numeric(dfxxx['ratingSignals'], errors='coerce')
dfxxx['stats.tipCount'] = pd.to_numeric(dfxxx['stats.tipCount'], errors='coerce')
dfxxx['price.tier'] = pd.to_numeric(dfxxx['price.tier'], errors='coerce')

In [83]:
px.bar(dfxxx,              
       x=dfxxx['name'],         
       y=dfxxx['rating'],   
       title=f"Rating",
       color_discrete_sequence=["blueviolet"], 
       height=500,
       width=800)

In [84]:
import plotly.graph_objects as go
fig = go.Figure(data=[
    go.Bar(name='Likes Count', x=dfxxx['name'], y=dfxxx['likes.count']),
    go.Bar(name='Rating Signals', x=dfxxx['name'], y=dfxxx['ratingSignals']),
    go.Bar(name='Tips Count', x=dfxxx['name'], y=dfxxx['stats.tipCount'])
])

fig.update_layout(barmode='group')
fig.show()

In [85]:
px.bar(dfxxx,              
       x=dfxxx['name'],         
       y=dfxxx['createdAt'],   
       title=f"Time on Foursquare",
       color_discrete_sequence=["blueviolet"], 
       height=500,
       width=800)

In [86]:
colors=dfxxx['price.tier']

px.bar(dfxxx,              
       x=dfxxx['name'],         
       y=dfxxx['ratingSignals'],   
       title=f"Traffic vs Price",
       height=500,
       width=800,
       color=colors,
      ) 
       


In [87]:
fig = go.Figure(data=[
    go.Bar(name='Price Tier', x=dfxxx['name'], y=dfxxx['price.tier']),
    go.Bar(name='Rating', x=dfxxx['name'], y=dfxxx['rating'])
])

fig.update_layout(barmode='group')
fig.show()

In [89]:

sweet_report = sv.analyze(dfxxx)
sweet_report.show_html('sweet_report.html')

                                             |          | [  0%]   00:00 -> (? left)

Report sweet_report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


## Conclusion

___

It looks like University park may be a great place to start a Mexican Restaraunt! There are only 3 in the neighborhood none of them appear to have good ratings. This could indicate that there is a demand. People seem to want this type of food to keep 3 venues open. There also seems to be opportunity (since the ratings and number of likes are low) to fill that demand with something that people will enjoy. Further market research is still needed but this area looks promising!